http://localhost:8070

In [7]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0",
    "com.clickhouse:clickhouse-http-client:0.6.0",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         # .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         # .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.0.142") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         # .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         # .config("spark.clickhouse.read.compression.codec", "lz4")
         # .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         .config("spark.clickhouse.write.batchSize", "10000000")
         # .config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         # .config("spark.clickhouse.options.socket_timeout", "600000000")
         # .config("spark.clickhouse.options.connection_timeout", "600000000")
         # .config("spark.clickhouse.options.query_timeout", "600000000")
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         .config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

DataFrame[]

In [8]:
df.show()

+---+--------+-----------+-------+--------+---------+-------------------+-------------+---------+--------------+--------------+-----------+--------+-----+-----------+-------------+--------+
| id|Location|     Region|Country|Latitude|Longitude|          Localtime|Temperature_C|Condition|Wind_Speed_kph|Wind_Direction|Pressure_mb|Humidity|Cloud|FeelsLike_C|Visibility_km|UV_Index|
+---+--------+-----------+-------+--------+---------+-------------------+-------------+---------+--------------+--------------+-----------+--------+-----+-----------+-------------+--------+
|  0|  Moscow|Moscow City| Russia|   55.75|    37.62|2024-04-28 00:15:00|          8.0|      Fog|          11.2|           WSW|        0.0|      93|    0|        6.4|         10.0|     1.0|
|  1|  Moscow|Moscow City| Russia|   55.75|    37.62|2024-04-28 01:24:00|          8.0|     Mist|          11.2|             W|        0.0|      87|    0|        6.4|         10.0|     1.0|
|  2|  Moscow|Moscow City| Russia|   55.75|    37.

In [2]:
spark.sql('show databases').show()

+------------------+
|         namespace|
+------------------+
|INFORMATION_SCHEMA|
|           default|
|    first_database|
|information_schema|
|            system|
+------------------+



In [3]:
spark.sql('CREATE DATABASE first_database').show()

AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `first_database` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [4]:
df = spark.read.csv('/projects/app/stations.csv',header=True,inferSchema=True)

In [2]:
df = spark.sql('''SELECT * FROM first_database.WeatherData''')

In [5]:

(
    df
    .write
    .mode("append")
    .format("jdbc")
    .option("url", "jdbc:clickhouse://34.118.0.142:9000/")
    .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
    .option("user", "default")
    .option("password", "1234")
    .option("dbtable", "first_database.WeatherDataSpark")
    .option("batchsize", "10000000")
    .option("numPartitions", cpu)
    .option("truncate", "true")
    .option("isolationLevel", "NONE")
    .save()
)


Py4JJavaError: An error occurred while calling o94.save.
: com.github.housepower.exception.InvalidValueException: Connection is not support
	at com.github.housepower.jdbc.ClickhouseJdbcUrlParser.parseJdbcUrl(ClickhouseJdbcUrlParser.java:56)
	at com.github.housepower.settings.ClickHouseConfig$Builder.withJdbcUrl(ClickHouseConfig.java:355)
	at com.github.housepower.jdbc.ClickHouseDriver.connect(ClickHouseDriver.java:48)
	at com.github.housepower.jdbc.ClickHouseDriver.connect(ClickHouseDriver.java:26)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.exe

In [3]:
df.createOrReplaceTempView('df_sql')
spark.sql("""
INSERT INTO first_database.WeatherDataSpark
SELECT * FROM df_sql
""")

24/04/28 08:55:57 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 1]
java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(WriteToDataSourceV2Exec.scala:491)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.$anonfun$writeWithV2$2(WriteToDataSourceV2Exec.scala:388)
	at org.apa

Py4JJavaError: An error occurred while calling o57.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (ae0a349aa8da executor driver): java.lang.NoSuchMethodError: 'org.apache.arrow.vector.types.pojo.Schema org.apache.spark.sql.util.ArrowUtils$.toArrowSchema(org.apache.spark.sql.types.StructType, java.lang.String)'
	at org.apache.spark.sql.clickhouse.SparkUtils$.toArrowSchema(SparkUtils.scala:28)
	at xenon.clickhouse.write.format.ClickHouseArrowStreamWriter.<init>(ClickHouseArrowStreamWriter.scala:31)
	at xenon.clickhouse.write.ClickHouseBatchWrite.createWriter(ClickHouseWrite.scala:73)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run(WriteToDataSourceV2Exec.scala:436)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.run$(WriteToDataSourceV2Exec.scala:425)
	at org.apache.spark.sql.execution.datasources.v2.DataWritingSparkTask$.run(Wri

In [ ]:
spark.sql("""
SELECT * FROM default.input_test_table
""").show()